Como en el enunciado de la tarea, se usará .to_crs(32719) para convertir todos los datos al mismo sistema coordenado de referencia, dejándolo por defecto me dio problemas.

3.1 Construya una nueva capa vectorial con un único polígono que encierre toda el área de estudio. Es decir, el polígono más pequeño que contenga a todas las viviendas catastradas [3pts].

In [77]:
import geopandas as gpd

# Cargamos el shapeile de las edificaciones.
edificaciones = gpd.read_file("Edificaciones.shp").to_crs(32719)

# Construimos una capa vectorial con un único polígono del área de estudio.
area_estudio = edificaciones.geometry.union_all().convex_hull

# Convertimos el area de estudio a GDF y lo almacenamos en una variable.
area_gdf = gpd.GeoDataFrame(geometry=[area_estudio], crs=edificaciones.crs)

# Guardamos la capa en un nuevo archivo, que posteriormente usaremos
area_gdf.to_file("area_estudio.shp")

Ahora usamos la librería rasterio para recortar el DEM

In [78]:
!pip install rasterio

3.2 Obtenga un DEM de la zona de estudio, y genere un raster de pendientes para el terreno. Indique claramente la fuente y link de descarga del DEM utilizado [3pts]:

Se ha utilizado el DEM Alos Palsar Región de Valparaíso entregada por el sitio web geoportal.cl:
https://www.geoportal.cl/geoportal/catalog/35435/DEM%20Alos%20Palsar%20Regi%C3%B3n%20de%20Valpara%C3%ADso

https://www.geoportal.cl/geoportal/catalog/download/b538717f-cc50-3c8e-bd28-2aff59d4f4e5

In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Ruta del archivo DEM de valpo, formato jp2.
dem_path_valpo = "5.jp2"

# Visualizamos el DEM para mayor seguridad
with rasterio.open(dem_path_valpo) as src:
    dem = src.read(1)  # Leer el primer dato de elevación
    plt.imshow(dem, cmap="terrain")
    plt.colorbar(label="Elevación")
    plt.title("Digital Elevation Model - DEM")
    plt.show()


In [ ]:
# Recorttamos el DEM usando el polígono anteriormente almacenado para
# el área de estudio.
from rasterio.mask import mask

# Recortamos el DEM en relación al área d estudio
with rasterio.open(dem_path_valpo) as src:
    dem_recortado, transform = mask(src, area_gdf.geometry, crop=True)

# Visualizamos el DEM recortado
plt.imshow(dem_recortado[0], cmap="terrain")
plt.colorbar(label="Elevación (m)")
plt.title("DEM Recortado al polígono de estudio")
plt.show()


3.3 Para cada vivienda, determine [8 puntos]:

    La distancia al grifo más cercano (dist_grifo)
    La distancia a la vía de acceso más cercana (dist_calle)
    Si está o no dentro del TO de la empresa sanitaria (in_TO)
    La pendiente del terreno (slope).

Para este punto, en las celdas de más abajo se almacenarán en variables los datos solicitados:

In [ ]:
# Aquí, usamos GDAL de osgeo, para abrir el raster de elevación (DEM), y
# calcular su pendiente, que luego almacenamos en slope_path
from osgeo import gdal

# Abrimos el DEM con GDAL
dem_ds = gdal.Open(dem_path_valpo)

# Calculamos la pendiente
slope_path = "slope.tif"
gdal.DEMProcessing(slope_path, dem_ds, "slope")

print(f"Raster de pendientes guardado en el archivo: {slope_path}")

Ahora, ya tenemos el DEM cargado, recortado al área de estudio, y hemos inalmente generado el raster de pendientes calculado!

In [ ]:
import numpy as np

# Revisamos la estructura del GeoDataFrame edificaciones antes de comenzar.
print(edificaciones.head())

# Cargamos los shapefiles de los grifos y lo pasamos a GeoDataFrame, con el CRS adecuado:
grifos = gpd.read_file("grifos_nacionales_2022.shp").to_crs(32719)

# Calculamos la distancia al grifo más cercano con una función de orden superior lambda,
# y agregamos esa nueva columna al GeoDataFrame posteriormente.
dist_grifo_min_values = edificaciones.geometry.apply(lambda x: grifos.geometry.distance(x).min)

Es importante destacar que en la celda anterior se hizo una suposición importante: se han tomado los grifos nacionales registrados en el año 2022.

Ahora, calculamos la distancia a la vía de acceso más cercana\

In [ ]:
# Cargar el shapefile de las calles
calles_gdf = gpd.read_file("Eje_Vial.shp").to_crs(32719)

# Igual que en la celda anterior, calculamos la distancia mínima entre cada edificación y
# las calles aledañas o cercanas, para agregar la nueva col "dist_calle_min" posteriormente
dist_calle_min_values = edificaciones.geometry.apply(lambda x: calles_gdf.geometry.distance(x).min())

Luego, verificamos si está dentro del Territorio Operacional (TO) de la empresa sanitaria en cuestión.

In [ ]:
# Cargamos el shapefile del TO (Territorio Operacional)
to_area = gpd.read_file("TO_202312_01032024_v1.shp").to_crs(32719)

# Verificar si cada vivienda está dentro del TO
in_to_values = edificaciones.geometry.apply(lambda x: to_area.contains(x).any())

A partir del DEM (raster), calculamos con GDAL las pendientes:

In [ ]:
from osgeo import gdal

# Abrimos el DEM y lo almacenamos en una variable
dem_ds = gdal.Open("Rocuant.tif")

# Procesamos el raster de las pendientes que anteriormente habíamos guardado en slope_path:
gdal.DEMProcessing(slope_path, dem_ds, "slope")

# Cargamos el raster de las pendientes
slope_dataset = gdal.Open(slope_path)
slope_array = slope_dataset.ReadAsArray()

# Ahora, necesitamos obtener la pendiente para cada vivienda, las que podemos conseguir mediante
# las coordenadas de cada vivienda que ya tenemos!

# Hay que revisar bien, pero de momento suponemos que las viviendas están bien alineadas con la resolución
# del DEM, que puede diferir, como también el sistema de referencia de coordenadas utilizado (CRS)

# Obtenemos la información de GeoTransform, que es una tupla con la información para poder
# mapear desde pixeles a coordenadas geográficas (proyección).
geotransform = slope_dataset.GetGeoTransform()

# Obtenemos los valores de la tupla GeoTransform y almacenamos sus valores:
top_left_x = slope_dataset.GetGeoTransform()[0]  # Coordenada X de la esquina superior izquierda
pixel_width = slope_dataset.GetGeoTransform()[1]  # Ancho de la celda (pxiel)
rotation_x = slope_dataset.GetGeoTransform()[2]  # Rotación en la dirección X
top_left_y = slope_dataset.GetGeoTransform()[3]  # Coordenada Y de la esquina superior izquierda
rotation_y = slope_dataset.GetGeoTransform()[4]  # Rotación en la dirección Y
pixel_height = slope_dataset.GetGeoTransform()[5]  # Alto de la celda (pixel)

# Imprimimos los valores ordenados de GeoTransform para revisar que todo esté bien:
print("GeoTransform:")
print(f"  Coordenada X de la esquina superior izquierda: {top_left_x}")
print(f"  Coordenada Y de la esquina superior izquierda: {top_left_y}")

# Ya que estamos usando el CRS EPSG:32719, cada pixel tendrá un tamaño en relación
# a cada metro (la unidad de ese CRS):
print(f"  Tamaño de la celda (pixel) en la dirección X: {pixel_width}")
print(f"  Tamaño de la celda (pxiel) en la dirección Y: {pixel_height}")

# Si la rotación es 0, el raster está orientado de forma estándar, sin rotaciones que
# puedan complejizar el análisis.
print(f"  Rotación en la dirección X: {rotation_x}")
print(f"  Rotación en la dirección Y: {rotation_y}")

def get_slope_vivienda(vivienda_geometry):
    centroid = vivienda_geometry.centroid # Obtener el centroide de la vivienda
    x_centroid_vivienda, y_centroid_vivienda = centroid.x, centroid.y

    # Calculo para la columna (col) y la fila (row) con los datos obtenidos del raster,
    # esto no siempre es necesario, pero aquí sirve para centrar y acomodar.
    col = int((x_centroid_vivienda - top_left_x) / pixel_width)
    row = int((y_centroid_vivienda - top_left_y) / pixel_height)

    # Nos aseguramosl que las coordenadas estén dentro del rango válido de filas y columnas
    # Sin estas lineas con numpy, me daba un IndexError
    row = np.clip(row, 0, slope_array.shape[0] - 1)
    col = np.clip(col, 0, slope_array.shape[1] - 1)

    # Devolver la pendiente en esa celda
    return slope_array[row, col]

# Calculamos la pendiente para cada vivienda, aplicando la función
# get_slope_vivienda para cada vivienda del GeoDataFrame edificaciones,
# agregando la nueva columna llamada "slope", para la pendiente (esta columna)
# se agregará al final, en una celda aparte.


Agregue cada una de estas variables como una columna adicional al GeoDataFrame creado a partir de Edificaciones.shp :

In [ ]:
# Con las variables calculadas y obtenidas más arriba, las agregamos como una nueva
# columna adicional al GeoDataFrame, como se indica en el enunciado:

edificaciones['in_TO'] = in_to_values
edificaciones['dist_calle_min'] = dist_calle_min_values
edificaciones['dist_grifo_min'] = dist_grifo_min_values

#Aquí agregamos la pendiente aplicando la función para cada edificación:
edificaciones['slope'] = edificaciones.geometry.apply(get_slope_vivienda)

# Revisamos que se hayan agregado las columnas al GDF:
print(edificaciones.head())

3.5 Utilizando todos los conjuntos de datos indicados en el punto 2, genere una única figura que incluya [6pts]:

    La imagen RGB de base.
    Las viviendas ubicadas en la zona de estudio, diferenciando con colores las viviendas con daño Total, Parcial o sin daño.
    Las calles.
    Los grifos.
    El territorio operacional.

Utilice buenas prácticas de gráficos como: etiquetas de ejes, título, leyenda, límites adecuados para ejes, etc.

In [ ]:
!pip install rioxarray

In [ ]:
import matplotlib.pyplot as plt
import rioxarray
import xarray as xr

# Definir las rutas de las imágenes
Rocuant_RGB_1 = 'Rocuant.tif'
Rocuant_RGB_2 = 'Rocuant2.tif'

# Abrir las imágenes utilizando rioxarray
rgb1 = rioxarray.open_rasterio(Rocuant_RGB_1)
rgb2 = rioxarray.open_rasterio(Rocuant_RGB_2)

# Crear una figura
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)

# Representar la primera imagen
rgb1.squeeze().plot.imshow(ax=ax, alpha=1)

# Superponer la segunda imagen con transparencia
rgb2.squeeze().plot.imshow(ax=ax, alpha=1)

# Añadir título y etiquetas
ax.set_title('Imágenes RGB del Incendio dadas en el enunciado')
ax.set_xlabel('Coordenadas Este (eje x)')
ax.set_ylabel('Coordenadas Norte (eje y)')

# Mostrar la figura
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import rioxarray
import matplotlib.patches as mpatches  # Lo usaremos para las áreas.
from matplotlib.lines import Line2D  # Lineas para la leyenda mostrada al final.

# Abrimos las imágenes RGB utilizando rioxarray
rgb1 = rioxarray.open_rasterio(Rocuant_RGB_1)
rgb2 = rioxarray.open_rasterio(Rocuant_RGB_2)

viviendas_gdf = edificaciones

# Hacemos un polígono que recorte!
total_bounds = viviendas_gdf.union_all().envelope

grifos_gdf = grifos.clip(total_bounds)
territorio_operacional_gdf = to_area.clip(total_bounds)
calles_gdf = calles_gdf.clip(total_bounds)

# Creamos una columna de colores para las viviendas según su daño
viviendas_gdf['color_dano'] = viviendas_gdf['N_daño'].map({
    'Total': 'red',  # Daño total: color rojo
    'Parcial': 'violet',  # Daño parcial: color naranja
    'Ninguno': 'green'  # Sin daño: color verde
})

# Figura
fig, ax = plt.subplots(figsize=(10, 10))

# Mostrar la imagen RGB de base con ambas imágenes
grb1_plot = rgb1.squeeze().plot.imshow(ax=ax, alpha=0.5)
grb2_plot = rgb2.squeeze().plot.imshow(ax=ax, alpha=0.5)

# Dibujar las viviendas
grifos_gdf.plot(ax=ax, markersize=10, color='blue', marker='*')
viviendas_gdf.plot(ax=ax, color=viviendas_gdf['color_dano'], markersize=10)
calles_gdf.plot(ax=ax, color='cyan')
territorio_operacional_gdf.boundary.plot(ax=ax, color='blue', lw=1)

# Etiquetas y título
ax.set_title('Mapa de Daños y Elementos en la Zona de Estudio')
ax.set_xlabel('Coordenadas Este (X)')
ax.set_ylabel('Coordenadas Norte (Y)')

# Creamos las entradas del gráfico
handles = [
    mpatches.Patch(color='red', label='Daño Total'),
    mpatches.Patch(color='violet', label='Daño Parcial'),
    mpatches.Patch(color='green', label='Sin Daño'),
    Line2D([0], [0], color='cyan', lw=2, label='Calles'),
    Line2D([0], [0], color='blue', marker='*', markersize=10, linestyle='None', label='Grifos'),
    Line2D([0], [0], color='blue', lw=1, label='Territorio Operacional')
]

# Leyendas
ax.legend(handles=handles)

plt.show()


3.6 Analice y comente: ¿identifica alguna relación entre la ubicación de las viviendas quemadas, y su posición relativa a grifos, redes de agua y vías de acceso? Genere uno o más gráficos simples (por ejemplo, gráficos de barras o dispersión) que soporten su análisis [4pts].

Parece ser que el incendio se propagó rápido o sin ser tan determinado por las vías de acceso y la cercanía a grifos, porque hay una gran cantidad de viviendas muy cerca de vías de acceso y grifos que resultaron afectadas. Sin embargo, esto no indica mucho por sí mismo.

Los gráficos y el análisis más completo que apoyan este punto están más abajo.

In [ ]:
import matplotlib.pyplot as plt
import rioxarray
import geopandas as gpd
import seaborn as sns
import numpy as np
from shapely.ops import nearest_points

# Cargamos las imágenes con rioxarray, en variables.
rgb1 = rioxarray.open_rasterio(Rocuant_RGB_1)
rgb2 = rioxarray.open_rasterio(Rocuant_RGB_2)

# Recorte de las capas para que solo se muestren en el área de las viviendas
total_bounds = viviendas_gdf.union_all().envelope
grifos_gdf = grifos_gdf.clip(total_bounds)
calles_gdf = calles_gdf.clip(total_bounds)

# Creamos columnas con colores segun el daño de cada vivienda, para luego
# usarlos en la gráfica:
viviendas_gdf['color_dano'] = viviendas_gdf['N_daño'].map({
    'Total': 'red',  # Daño total: color rojo
    'Parcial': 'violet',  # Daño parcial: color naranja
    'Ninguno': 'green'  # Sin daño: color verde
})

# Definir una función para calcular la distancia más cercana
def calcular_distancia_cercana(vivienda, puntos_geometria):
    nearest_geom = nearest_points(vivienda, puntos_geometria)
    return vivienda.distance(nearest_geom[1])

# Calcular la distancia de cada vivienda a los grifos y las vías
grifos_geom = grifos_gdf.geometry.union_all()  # Unificamos la geometría de los grifos
calles_geom = calles_gdf.geometry.union_all()  # Unificamos la geometría de las calles

# Lo calculamos nuevamente por facilidad, aunque incluya mayor cómputo
viviendas_gdf['distancia_grifo'] = viviendas_gdf.geometry.apply(lambda x: calcular_distancia_cercana(x, grifos_geom))
viviendas_gdf['distancia_via'] = viviendas_gdf.geometry.apply(lambda x: calcular_distancia_cercana(x, calles_geom))

# Histograma de distancias a los grifos con seaborn
plt.figure(figsize=(10, 6))
sns.histplot(viviendas_gdf['distancia_grifo'], kde=True, color='red', bins=30)
plt.title('Distribución de las Distancias a los Grifos')
plt.xlabel('Distancia a los Grifos (m)')
plt.ylabel('Número de Viviendas')
plt.show()

# Histograma de distancias a las vías de acceso con seabor
plt.figure(figsize=(10, 6))
sns.histplot(viviendas_gdf['distancia_via'], kde=True, color='cyan', bins=30)
plt.title('Distribución de las Distancias a las Vías de Acceso')
plt.xlabel('Distancia a las Vías de Acceso (m)')
plt.ylabel('Número de Viviendas')
plt.show()

# Gráfico de dispersión de distancias a las vías - distancias a los grifos
plt.figure(figsize=(10, 6))

# Usamos los colores correspondientes según el daño
# aplicando los colores a  la leyenda
sns.scatterplot(x=viviendas_gdf['distancia_via'], y=viviendas_gdf['distancia_grifo'],
                hue=viviendas_gdf['color_dano'],
                palette={'red': 'red', 'violet': 'violet', 'green': 'green'},
                alpha=0.65, legend=False) #False para personalizarn nuestra leyenda

# Títulos y etiquetas
plt.title('Distancias a Vías vs. Distancias a Grifos')
plt.xlabel('Distancia a las Vías de Acceso (m)')
plt.ylabel('Distancia a los Grifos (m)')

# Creaamos las leyendas del gráfico:
import matplotlib.patches as mpatches
red_patch = mpatches.Patch(color='red', label='Daño Total')
violet_patch = mpatches.Patch(color='violet', label='Daño Parcial')
green_patch = mpatches.Patch(color='green', label='Sin Daño')

plt.legend(handles=[red_patch, violet_patch, green_patch], title='Nivel de Daño')

plt.show()


### 3.7 Analice la ubicación de las viviendas afectadas por el incendio: ¿serán viviendas formales, o zonas de ocupación informal?
* Plantee y explique (en palabras) un método de análisis espacial que permita determinar tentativamente si una vivienda fue contruida formal o informalmente (es decir, sin permisos legales, o sin estar registrada por las autoridades). En caso de requerir otros conjuntos de datos, explique y obtengalos si es posible de alguna fuente oficial.
* Aplique su método de análisis a las zonas afectadas por los incendios de Valparaíso, y
*

Como método de análisis espacial para determinar tentativamente si una vivienda es informal o no, podemos utilizar los límites entregados el Territorio Operacional (TO), visualmente en la imagen construida arriba podemos ver las viviendas que están fuera del TO, pero también podemos mostrarlas recorriendo las el GeoDataFrame de viviendas.

Luego, en el último asterisco de 3.7, como parece estar el enunciado incompleto, interpreté como pude este último punto:

Aplicamos el método de análisis mencionado previamente:

In [ ]:
import pandas as pd

# Filtramos las viviendas afectadas total y parcialmente:
viviendas_afectadas = viviendas_gdf[viviendas_gdf['N_daño'].isin(['Total', 'Parcial'])].copy()

# Usamos .loc[] para evitar el error SettingWithCopyWarning
viviendas_afectadas.loc[:, 'dist_calle_min'] = pd.to_numeric(viviendas_afectadas['dist_calle_min'], errors='coerce')
viviendas_afectadas.loc[:, 'dist_grifo_min'] = pd.to_numeric(viviendas_afectadas['dist_grifo_min'], errors='coerce')

# Contamos las viviendas afectadas dentro y fuera del Territorio operacional:
conteo = viviendas_afectadas.groupby('in_TO')['OBJECTID'].count()

# Calcular porcentajes
total_viviendas_afectadas = viviendas_afectadas.shape[0]
porcentajes = (conteo / total_viviendas_afectadas) * 100

# Crear tabla resumen
tabla_resumen = pd.DataFrame({
    'Viviendas Afectadas en total': conteo,
    'Porcentaje de viviendas afectadas': porcentajes,
    'Distancia Promedio a Calle': viviendas_afectadas.groupby('in_TO')['dist_calle_min'].mean(),
    'Distancia Promedio a Grifo': viviendas_afectadas.groupby('in_TO')['dist_grifo_min'].mean(),
})

# Ploteamos la visualización
plt.figure(figsize=(9, 5))

# Graficamos las columnas:
tabla_resumen.plot(kind='bar', ax=plt.gca(), colormap='viridis')

plt.title('Viviendas Afectadas')
plt.xlabel('Vivienda pertenece al TO? No / Sí')
plt.ylabel('Valores')

plt.show()


In [ ]:
# Creamos un nuevo dataframe para mostrarlo>
tabla_resumen = pd.DataFrame({
    'Viviendas Afectadas en total': conteo,
    'Porcentaje de viviendas afectadas': porcentajes
})

print("\nResumen de Viviendas Afectadas dentro del TO:\n")
print(tabla_resumen,'\n\n')

# Contamos las viviendas dentro y fuera del TO
conteo_TO = viviendas_gdf['in_TO'].value_counts()

print(f"\nConteo de Viviendas dentro y fuera del Territorio Operacional (TO):\n")
print(f"Viviendas dentro del TO: {conteo_TO.get(True, 'Error')}")
print(f"Viviendas fuera del TO: {conteo_TO.get(False, 'Error')}")

Como vemos, gran parte de las viviendas afectadas (total o parcialmente) pertenecen al territorio operacional, lo que no indica que dichas viviendas tengan mayor riesgo por el hecho de pertenecer, ya que la cantidad de viviendas pertenecientes al TO es mucho mayor (más del doble), como se puede apreciar arriba. Lo que sí podemos ver claramente es que la distancia de las viviendas fuera del TO a los grifos es, en promedio, mucho mayor, lo que claramente es un factor de riesgo ante este tipo de catástrofes.

Aquí, para un análisis más profundo, se deberían considerar todos los datos, como por ejemplo; si las viviendas estaban muy juntas entre ellas (esto ayuda a la propagación del fuego), el material de construcción, la pendiente del terreno, la orientación de las viviendas, entre muchos otros parámetros, pero todo este análisis mayor escapa a esta tarea.

Referencias: para esta tarea, se han utilizado como referencia los códigos de las ayudantías, de este año y de Canvas del año pasado, pero mayormente el código ya entregado en parte por el enunciado de la tarea.

He tomado ejemplos de la documentación de oficial de seaborn y matplotlib para las gráficas y leyendas.

Para agilizar el cálculo de los porcentajes de la última y penúltima celda, se utilizó ChatGPT, lo mismo con la implementación de la linea .loc[] para evitar el error SettingWithCopyWarning.